In [11]:
import os
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

In [12]:
dos_path = r"/Users/tharhtet/Documents/test3/wizard_of_oz/pg54.txt"
loader = TextLoader(dos_path)
documents = loader.load()
print(type(documents))

<class 'list'>


In [13]:
text_splitter = CharacterTextSplitter(chunk_size = 1000 , chunk_overlap=0)
docs = text_splitter.split_documents(documents)

Created a chunk of size 1184, which is longer than the specified 1000
Created a chunk of size 1045, which is longer than the specified 1000
Created a chunk of size 1132, which is longer than the specified 1000
Created a chunk of size 1674, which is longer than the specified 1000


In [14]:
len(docs)

298

In [15]:
#openai_embeddings = OpenAIEmbeddings()
#db = FAISS.from_documents(docs, openai_embeddings)

In [16]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

modelPath = "sentence-transformers/all-MiniLM-l6-v2"
#modelPath = "sentence-transformers/all-mpnet-base-v2"

model_kwargs = {"device" : 'mps'}
encoding_kwargs = {"normalize_embeddings": False}
embeddings = HuggingFaceBgeEmbeddings(
    model_name = modelPath,
    model_kwargs = model_kwargs,
    encode_kwargs=encoding_kwargs
)

/opt/anaconda3/envs/ths/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [17]:
db = FAISS.from_documents(docs, embeddings)

In [18]:
question = "Where are you from, Dorothy Gale?"
retriever = db.as_retriever()
new_docs =retriever.invoke(question)

In [19]:
for cur_ in new_docs:
    print(cur_.page_content)
    print("######")

“Oh!” said Jack. Then, after a pause, he asked: “Did you say a Tin
Woodman rules the Winkies?”

“Yes; he was one of those who helped Dorothy to destroy the Wicked
Witch of the West, and the Winkies were so grateful that they invited
him to become their ruler,—just as the people of the Emerald City
invited the Scarecrow to rule them.”

“Dear me!” said Jack. “I’m getting confused with all this history. Who
is the Scarecrow?”

“Another friend of Dorothy’s,” replied Tip.

“And who is Dorothy?”

“She was a girl that came here from Kansas, a place in the big, outside
World. She got blown to the Land of Oz by a cyclone, and while she was
here the Scarecrow and the Tin Woodman accompanied her on her travels.”

“And where is she now?” inquired the Pumpkinhead.

“Glinda the Good, who rules the Quadlings, sent her home again,” said
the boy.

“Oh. And what became of the Scarecrow?”

“I told you. He rules the Emerald City,” answered Tip.
######
“Being Thoroughly Educated, and knowing what is requir

In [21]:
save_path = "/Users/tharhtet/Documents/github/RAG-for-NPC-Game-Characters/corpus/wizard_of_oz/faiss_index"
db.save_local(save_path)